# **HW1: Regression** 
In *assignment 1*, you need to finish:

1.  Basic Part: Implement the regression model to predict the number of dengue cases


> *   Step 1: Split Data
> *   Step 2: Preprocess Data
> *   Step 3: Implement Regression
> *   Step 4: Make Prediction
> *   Step 5: Train Model and Generate Result

2.  Advanced Part: Implementing a regression model to predict the number of dengue cases in a different way than the basic part

# 1. Basic Part (60%)
In the first part, you need to implement the regression to predict the number of dengue cases

Please save the prediction result in a csv file **hw1_basic.csv**


## Import Packages

> Note: You **cannot** import any other package in the basic part

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import math
import random

## Global attributes
Define the global attributes

In [31]:
input_dataroot = 'hw1_basic_input.csv' # Input file named as 'hw1_basic_input.csv'
output_dataroot = 'hw1_basic.csv' # Output file will be named as 'hw1_basic.csv'

input_datalist =  [] # Initial datalist, saved as numpy array
output_datalist =  [] # Your prediction, should be 10 * 4 matrix and saved as numpy array
             # The format of each row should be ['epiweek', 'CityA', 'CityB', 'CityC']

You can add your own global attributes here


In [190]:
train_dataset_list = []
valid_dataset_list = []
processed_dataset_list = [[],[],[]]

## Load the Input File
First, load the basic input file **hw1_basic_input.csv**

Input data would be stored in *input_datalist*

In [46]:
# Read input csv to datalist
with open(input_dataroot, newline='') as csvfile:
  input_datalist = np.array(list(csv.reader(csvfile)))

## Implement the Regression Model

> Note: It is recommended to use the functions we defined, you can also define your own functions


### Step 1: Split Data
Split data in *input_datalist* into training dataset and validation dataset 



In [84]:
def SplitData():
    global train_dataset_list
    global valid_dataset_list
    data_row = np.size(input_datalist,0) - 11   #number of rows without title(1) and data we want to predict(10)
    num_train = int(data_row * 0.7)             #number of dataset for training
    num_valid = data_row - num_train         #number of dataset for validation
    train_dataset_list = np.array(input_datalist[1:num_train+1])
    valid_dataset_list = input_datalist[num_train+1:num_train+num_valid+1]

### Step 2: Preprocess Data
Handle the unreasonable data
> Hint: Outlier and missing data can be handled by removing the data or adding the values with the help of statistics  

In [195]:
def PreprocessData():

    global processed_dataset_list
    
    for i in [1,2,3]:
        
        #detect empty input in column with index i in the dataset(false means empty in mask)
        mask = np.isin(train_dataset_list[:,i], [''], invert = True)

        #remove rows with empty input in dataset
        dataset_remove_empty = train_dataset_list[mask]

        #change the data type in dataset, and calculate the absolute value of each element's Z score
        temp = dataset_remove_empty[:,i].astype(np.double)
        temp_Z_score_abs = np.absolute((temp - np.average(temp))/np.std(temp))

        #change the value to integer which help us to identify outliers
        mask_t = np.isin(temp_Z_score_abs.astype(np.int64),[0], invert = False)
        
        #store only the temperature, data and number of cases in the list
        processed_dataset_list[i-1] = dataset_remove_empty[mask_t].transpose()[np.array([0,i,i+3])].transpose()

### Step 3: Implement Regression
> Hint: You can use Matrix Inversion, or Gradient Descent to finish this part




In [40]:
def Regression():
    

IndentationError: expected an indented block (2074696770.py, line 2)

### Step 4: Make Prediction
Make prediction of testing dataset and store the value in *output_datalist*

In [ ]:
def MakePrediction():
  

### Step 5: Train Model and Generate Result

> Notice: **Remember to output the coefficients of the model here**, otherwise 5 points would be deducted
* If your regression model is *3x^2 + 2x^1 + 1*, your output would be: 
```
3 2 1
```





In [196]:
SplitData()
PreprocessData()
print("cityA")
print(processed_dataset_list[0])
print("cityB")
print(processed_dataset_list[1])
print("cityC")
print(processed_dataset_list[2])

cityA
[['202001' '21.48' '147']
 ['202003' '24.66' '198']
 ['202004' '23.89' '180']
 ['202005' '22.85' '162']
 ['202006' '27.49' '127']
 ['202008' '26.2' '99']
 ['202009' '23.51' '94']
 ['202010' '27.67' '67']
 ['202011' '25.44' '78']
 ['202012' '27.54' '70']
 ['202013' '27.78' '90']
 ['202014' '26.63' '84']
 ['202015' '26.22' '83']
 ['202022' '26.11' '60']
 ['202023' '24.65' '62']
 ['202024' '20.7' '62']
 ['202025' '20.69' '45']
 ['202026' '25.28' '55']
 ['202027' '25.35' '61']
 ['202028' '28.16' '50']
 ['202029' '23.7' '44']
 ['202030' '19.67' '59']
 ['202031' '27.55' '44']
 ['202032' '28.77' '38']
 ['202033' '23.81' '44']
 ['202034' '24.41' '44']
 ['202035' '24.61' '46']
 ['202036' '27.31' '39']
 ['202038' '27.64' '44']
 ['202040' '22.74' '49']
 ['202041' '22.56' '40']
 ['202042' '27.25' '46']
 ['202044' '23.93' '37']
 ['202045' '25.92' '48']
 ['202046' '28.59' '59']
 ['202047' '24.09' '49']
 ['202048' '22.84' '66']
 ['202049' '24.47' '43']
 ['202050' '23.53' '53']
 ['202051' '26.23

## Write the Output File
Write the prediction to output csv
> Format: 'epiweek', 'CityA', 'CityB', 'CityC'

In [ ]:
with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
  writer = csv.writer(csvfile)
  for row in output_datalist:
    writer.writerow(row)

# 2. Advanced Part (35%)
In the second part, you need to implement the regression in a different way than the basic part to help your predictions for the number of dengue cases

We provide you with two files **hw1_advanced_input1.csv** and **hw1_advanced_input2.csv** that can help you in this part

Please save the prediction result in a csv file **hw1_advanced.csv** 


# Report *(5%)*

Report should be submitted as a pdf file **hw1_report.pdf**

*   Briefly describe the difficulty you encountered 
*   Summarize your work and your reflections 
*   No more than one page






# Save the Code File
Please save your code and submit it as an ipynb file! (**hw1.ipynb**)